In [20]:
import duckdb
import pandas as pd

# Import for checking dtypes
from pandas.api.types import (
    is_bool_dtype,
    is_float_dtype,
    is_integer_dtype,
    is_string_dtype,
    is_datetime64_any_dtype
)

from bigdata_cleaning.core import check_data, fix_data

# DuckDB Connection
con = duckdb.connect()

# Data path
DATA_PATH = "../data/deutsche_bahn_data/monthly_processed_data/data-2024-10.parquet"

# Load dataset
df = pd.read_parquet(DATA_PATH)
con.register("data_table", df) # Convert to table

In [4]:
print(f"\nDatensatz geladen:")
print(f"  Zeilen: {len(df):,}")
print(f"  Spalten: {len(df.columns)}")
print(f"\nSpalten: {df.columns.tolist()}")


Datensatz geladen:
  Zeilen: 1,984,484
  Spalten: 16

Spalten: ['station_name', 'xml_station_name', 'eva', 'train_name', 'final_destination_station', 'delay_in_min', 'time', 'is_canceled', 'train_type', 'train_line_ride_id', 'train_line_station_num', 'arrival_planned_time', 'arrival_change_time', 'departure_planned_time', 'departure_change_time', 'id']


# 🔍 Teil 1: Exploration & Erste Checks

In [5]:
result = con.execute(f"""
SELECT
    COUNT(*) as total_rows,
    COUNT(DISTINCT station_name) as unique_stations,
    COUNT(DISTINCT train_name) as unique_trains,
    MIN(time) as first_timestamp,
    MAX(time) as last_timestamp,
    COUNT(DISTINCT train_type) as train_types
FROM '{DATA_PATH}'
""").fetchone()

print("\n📊 DATEN-ÜBERSICHT:")
print(f"  Zeilen gesamt: {result[0]:,}")
print(f"  Unique Bahnhöfe: {result[1]:,}")
print(f"  Unique Züge: {result[2]:,}")
print(f"  Zeitraum: {result[3]} bis {result[4]}")
print(f"  Zugtypen: {result[5]}")


📊 DATEN-ÜBERSICHT:
  Zeilen gesamt: 1,984,484
  Unique Bahnhöfe: 108
  Unique Züge: 1,548
  Zeitraum: 2024-10-01 00:00:00 bis 2024-10-31 23:59:00
  Zugtypen: 53


In [7]:
# Get schema from columns
data_schema = df.dtypes
print(data_schema)

station_name                         object
xml_station_name                     object
eva                                  object
train_name                           object
final_destination_station            object
delay_in_min                          int32
time                         datetime64[ns]
is_canceled                            bool
train_type                           object
train_line_ride_id                   object
train_line_station_num                int64
arrival_planned_time         datetime64[ns]
arrival_change_time          datetime64[ns]
departure_planned_time       datetime64[ns]
departure_change_time        datetime64[ns]
id                                   object
dtype: object


# 🕵️ Teil 2: Datenqualitäts-Checks

## 2.1 COMPLETENESS (Vollständigkeit)

**Frage dich:**
- Gibt es fehlende Werte (NULL)?
- In welchen Spalten?
- Wie viele?

In [21]:
# Check for missing data based on dtype
for col, dtype in data_schema.items():
    print(f"Column: {col}, Dtype: {dtype}")
    if is_bool_dtype(dtype):
        # Boolean: only NULL
        cond = f"{col} IS NULL"

    elif is_float_dtype(dtype):
        # Floats: NULL or NaN
        cond = f"{col} IS NULL OR isnan({col})"

    elif is_integer_dtype(dtype):
        # Integers: NULL only (no NaN possible)
        cond = f"{col} IS NULL"

    elif is_string_dtype(dtype):
        # Strings: NULL, empty, spaces, literal 'NaN'
        cond = f"{col} IS NULL OR trim({col}) = '' OR lower(trim({col})) = 'nan'"

    elif is_datetime64_any_dtype(dtype):
        # Datetime: NULL only
        cond = f"{col} IS NULL"

    else:
        cond = f"{col} IS NULL"

    query = f"""
        SELECT
            COUNT(*) as total,
            SUM(CASE WHEN {cond} THEN 1 ELSE 0 END) AS missing
        FROM data_table
    """

    total, missing = con.execute(query).fetchone()
    pct = round((missing / total) * 100, 2) if total else 0.0
    print(f"{col}: missing = {missing} ({pct}%)")

Column: station_name, Dtype: object
station_name: missing = 35757 (1.8%)
Column: xml_station_name, Dtype: object
xml_station_name: missing = 0 (0.0%)
Column: eva, Dtype: object
eva: missing = 0 (0.0%)
Column: train_name, Dtype: object
train_name: missing = 0 (0.0%)
Column: final_destination_station, Dtype: object
final_destination_station: missing = 4478 (0.23%)
Column: delay_in_min, Dtype: int32
delay_in_min: missing = 0 (0.0%)
Column: time, Dtype: datetime64[ns]
time: missing = 0 (0.0%)
Column: is_canceled, Dtype: bool
is_canceled: missing = 0 (0.0%)
Column: train_type, Dtype: object
train_type: missing = 0 (0.0%)
Column: train_line_ride_id, Dtype: object
train_line_ride_id: missing = 989578 (49.87%)
Column: train_line_station_num, Dtype: int64
train_line_station_num: missing = 0 (0.0%)
Column: arrival_planned_time, Dtype: datetime64[ns]
arrival_planned_time: missing = 441997 (22.27%)
Column: arrival_change_time, Dtype: datetime64[ns]
arrival_change_time: missing = 441914 (22.27%)
Co

===================================================================
### PROBLEM 1: Fehlende Werte
===================================================================

KATEGORIE: [Completeness]

BESCHREIBUNG:
Einige Columns haben Null-Werte, obwohl diese nicht erlaubt sind.

BETROFFENE DATEN:
- Spalte: station_name
- Anzahl Zeilen betroffen: 35757 (1.8%)
- Schweregrad: [Niedrig]

BEWEIS (SQL/Code):
- Siehe oben

AUSWIRKUNG:
- Die Zeilen würden bei Analysen automatisch ausgeschlossen werden

FIX-STRATEGIE:
- Zeilen entfernen

## 2.2 VALIDITY (Gültigkeit)

**Frage dich:**
- Gibt es unmögliche Werte?
- Negative Werte wo sie nicht sein sollten?
- Werte außerhalb des gültigen Bereichs?

In [29]:
query = f"""
SELECT
    COUNT(delay_in_min),
	MIN(delay_in_min) as min_delay,
    MAX(delay_in_min) as max_delay,
    AVG(delay_in_min) as avg_delay,
    COUNT(CASE WHEN delay_in_min < 0 THEN 1 END) as negative_delays,
    COUNT(CASE WHEN delay_in_min > 120 THEN 1 END) as extreme_delays
FROM data_table
"""

result = con.execute(query).fetchone()
print(result)
print(f"{round((result[5]/result[0])*100, 2)}")

(1984484, -1432, 849, 3.7567483537282236, 46235, 1350)
0.07


===================================================================
### PROBLEM 2: Unmögliche Werte
===================================================================

KATEGORIE: [Validity]

BESCHREIBUNG:
- Einige Werte in der Spalte 'delay_in_min' sind unrealistisch niedrig (negativ) oder sehr hoch (849 min - aber noch möglich)

BETROFFENE DATEN:
- Spalte: [delay_in_min]
- Anzahl Zeilen betroffen:
  - Negative Werte: 46235 (2.33%)
  - Sehr hohe Extremwerte: 1350 (0.07%)
- Schweregrad: [Hoch]

BEWEIS (SQL/Code):
- Siehe oben

AUSWIRKUNG:
- Berechnungen über die Verspätung werden fehlerhaft

FIX-STRATEGIE:
- Entweder negative Verspätungen auf 0 setzen, oder Zeilen entfernen

## 2.3 CONSISTENCY (Konsistenz)

**Frage dich:**
- Sind die Daten konsistent formatiert?
- Gibt es Encoding-Probleme?
- Gibt es Widersprüche in den Daten?

In [49]:
# Encoding Probleme:

bad_char = "\uFFFD"

query = f"""
    SELECT
        DISTINCT station_name
    FROM data_table
    WHERE station_name LIKE '%{bad_char}%'  -- Zeichen die nicht richtig encodiert sind
    LIMIT 10
"""

result = con.execute(query).fetchone()
print(result)

# Check: Logik-Konsistenz
query = f"""
    SELECT
        station_name,
        train_name,
        is_canceled,
        delay_in_min
    FROM data_table
    WHERE is_canceled = True AND delay_in_min > 0
    LIMIT 10
"""

result = con.execute(query).fetchone()
print(result)


None
('Bruchsal', 'S 3', True, 69)


## 2.4 ACCURACY (Genauigkeit)

**Frage dich:**
- Gibt es Outliers?
- Sind die Werte realistisch?
- Gibt es statistische Anomalien?

In [50]:
# Check: Logik-Konsistenz
query = f"""
    SELECT
        station_name,
        train_name,
        delay_in_min,
        is_canceled,
        time
    FROM data_table
    WHERE delay_in_min > 300
    ORDER BY delay_in_min DESC
    LIMIT 10
"""

result = con.execute(query).fetchone()
print(result)


(None, 'Bus SEVS4', 849, False, datetime.datetime(2024, 10, 2, 3, 34))


## 2.5 UNIQUENESS (Eindeutigkeit)

**Frage dich:**
- Gibt es Duplikate?
- Sind IDs wirklich einzigartig?

In [55]:
string_cols = df.select_dtypes(include="object").columns

for col in string_cols:
    print("\n" + col)
    query = f"""
        SELECT
            COUNT(*) as total_rows,
            COUNT(DISTINCT {col}) as unique_ids,
            COUNT(*) - COUNT(DISTINCT {col}) as duplicates
        FROM data_table
    """
    result = con.execute(query).fetchone()
    print(f"Row count: {result[0]}")
    print(f"N duplicates: {result[2]} ({round((result[2]/result[0])*100, 2)})")
    print(f"N unique values: {result[1]}")


station_name
Row count: 1984484
N duplicates: 1984376 (99.99)
N unique values: 108

xml_station_name
Row count: 1984484
N duplicates: 1984343 (99.99)
N unique values: 141

eva
Row count: 1984484
N duplicates: 1984342 (99.99)
N unique values: 142

train_name
Row count: 1984484
N duplicates: 1982936 (99.92)
N unique values: 1548

final_destination_station
Row count: 1984484
N duplicates: 1983075 (99.93)
N unique values: 1409

train_type
Row count: 1984484
N duplicates: 1984431 (100.0)
N unique values: 53

train_line_ride_id
Row count: 1984484
N duplicates: 1951746 (98.35)
N unique values: 32738

id
Row count: 1984484
N duplicates: 0 (0.0)
N unique values: 1984484


===================================================================
### PROBLEM 4: Uniqueness
===================================================================

KATEGORIE: [Uniqueness]

BESCHREIBUNG:
- In vielen Kategorischen Columns gibt es eine kleine Anzahl von unique values (eva, train_name, final_destination_station, train_type, train_line_ride_id).
- Einige der Einträge können valide sein, aber es könnte auch anhand einer falschen Scheibweise liegen

BETROFFENE DATEN:
- Spalte: [eva, train_name, final_destination_station, train_type, train_line_ride_id]
- Anzahl Zeilen betroffen: Siehe oben
- Schweregrad: [Mittel]

BEWEIS (SQL/Code):
- Siehe Oben

AUSWIRKUNG:
- Zuordnung der Zugparameter klappt nicht mehr zuverlässig

FIX-STRATEGIE:
- Zuerst Schreibweise der Einträge überprüfen und gegenfalls anpassen (uFFFD check)
- Anschließend
  - Entweder alle unique values anzeigen lassen und schauen ob die strings valide sind
  - Oder alle unique values herausfiltern
    - Es gehen realtiv wenig Daten verloren
    - Kategorien mit nur einem Eintrag haben nicht geng datenpunkte um Rückschlüsse zu ermöglichen